In [1]:
from alice_blue import *
import time
import pandas as pd
import numpy as np
import datetime
import account_details

In [3]:
usr_name = account_details.usr_name
pwd = account_details.pwd
tfa = account_details.tfa
app_id = account_details.app_id
api_scrt = account_details.api_scrt

# redirect url = 'https://ant.aliceblueonline.com/plugin/callback'

In [4]:
access_token = AliceBlue.login_and_get_access_token(username= usr_name, password= pwd, twoFA=tfa, 
                                                    api_secret=api_scrt, app_id = app_id)
print(access_token)

5OoOVGxCzVavNyaDE-8nuFRo5tZAmE97CAL0pU8jiwo.T30FtUHnQOD2Uz84sDR3Vy_sLLYMMgHNhP_yrbbK3lU


In [5]:
alice = AliceBlue(username=usr_name, password=pwd, access_token=access_token)

In [6]:
# get balance
print('Available balance: ', alice.get_balance()['data']['cash_positions'][0]['available']['cashmarginavailable'])

Available balance:  208.38


In [6]:
# df_reliance = pd.DataFrame(columns=['timestamp', 'ltp', 'volume'])
# timestamp = []
# ltp = []
# volume = []

df_reliance = pd.DataFrame(columns=['timestamp', 'ltp', 'volume'])
timestamp_reliance = []
ltp_reliance = []
volume_reliance = []

df_hdfcbank = pd.DataFrame(columns=['timestamp', 'ltp', 'volume'])
timestamp_hdfcbank = []
ltp_hdfcbank = []
volume_hdfcbank = []

In [7]:
socket_opened = False
# ltp = []
def event_handler_quote_update(message):
#     print(f"quote update {message}")
#     print('time: ', datetime.datetime.fromtimestamp(message['exchange_time_stamp']).strftime('%Y-%m-%d %H:%M:%S'))
#     print('price: ', message['ltp'])

    if message['instrument'].symbol == 'RELIANCE':  
        # save data
        timestamp_reliance.append(datetime.datetime.fromtimestamp(message['exchange_time_stamp']).strftime('%Y-%m-%d %H:%M:%S'))
        ltp_reliance.append(message['ltp'])
        volume_reliance.append(message['volume'])
        
#         print('Reliance data: ')
#         print('LTP: ', message['ltp'])
#         print('Volume: ', message['volume'])
        
    if message['instrument'].symbol == 'HDFCBANK':  
        # save data
        timestamp_hdfcbank.append(datetime.datetime.fromtimestamp(message['exchange_time_stamp']).strftime('%Y-%m-%d %H:%M:%S'))
        ltp_hdfcbank.append(message['ltp'])
        volume_hdfcbank.append(message['volume'])
        
#         print('HDFC Bank data: ')
#         print('LTP: ', message['ltp'])
#         print('Volume: ', message['volume'])

    

def open_callback():
    global socket_opened
    socket_opened = True

alice.start_websocket(subscribe_callback=event_handler_quote_update,
                      socket_open_callback=open_callback,
                      run_in_background=True)
while(socket_opened==False):
    pass

# alice.subscribe(alice.get_instrument_by_symbol('NSE', 'RELIANCE'), LiveFeedType.MARKET_DATA)
# alice.subscribe(alice.get_instrument_by_symbol('NSE', 'Nifty Bank'), LiveFeedType.COMPACT)
# alice.subscribe(alice.get_instrument_by_symbol('NSE', 'RELIANCE'), LiveFeedType.COMPACT)
instrument = [alice.get_instrument_by_symbol('NSE', 'RELIANCE'), 
              alice.get_instrument_by_symbol('NSE', 'HDFCBANK')]

alice.subscribe(instrument, LiveFeedType.MARKET_DATA)


In [8]:
# while True:
#     if ((len(ltp) % 60) == 0):
#         df_reliance['timestamp'] = timestamp
#         df_reliance['ltp'] = ltp
#         df_reliance['volume'] = volume
#         df_reliance.to_csv('Reliance_24_march.csv', index=False)
#         print('Data saved')
#         print('Amount of data saved: ', len(ltp))

In [ ]:
from time import sleep
while(True):
    df_reliance = pd.DataFrame(columns=['timestamp', 'ltp', 'volume'])
    df_reliance['timestamp'] = timestamp_reliance
    df_reliance['ltp'] = ltp_reliance
    df_reliance['volume'] = volume_reliance
    df_reliance.to_csv('./Tick_data/Reliance_08_April.csv', index=False)
    print('Data saved')
    print('Amount of data saved: ', len(ltp_reliance))
    
    df_hdfcbank = pd.DataFrame(columns=['timestamp', 'ltp', 'volume'])
    df_hdfcbank['timestamp'] = timestamp_hdfcbank
    df_hdfcbank['ltp'] = ltp_hdfcbank
    df_hdfcbank['volume'] = volume_hdfcbank
    df_hdfcbank.to_csv('./Tick_data/Hdfcbank_08_April.csv', index = False)
    print('Data saved')
    print('Amount of data saved: ', len(ltp_hdfcbank))
    sleep(60)

Data saved
Amount of data saved:  30
Data saved
Amount of data saved:  30
Data saved
Amount of data saved:  89
Data saved
Amount of data saved:  90


In [13]:
# alice.unsubscribe(alice.get_instrument_by_symbol('NSE', 'Nifty Bank'), LiveFeedType.MARKET_DATA)

In [14]:
# alice.subscribe([alice.get_instrument_by_symbol('NSE', 'TATASTEEL'), alice.get_instrument_by_symbol('NSE', 'RELIANCE')], LiveFeedType.MARKET_DATA)